In [ ]:

import os

# Fix for Colab T4 + LLaMA + LoRA
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["ACCELERATE_DISABLE_RICH"] = "1"
os.environ["ACCELERATE_USE_CPU"] = "0"

# Disable SDPA (T4 cannot train with SDPA)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_SDPA_ENABLE"] = "0"
os.environ["ATTN_BACKEND"] = "EAGER"

print("Environment variables set.")

Environment variables set.


In [ ]:
!pip install -U "transformers>=4.44.0" datasets accelerate peft evaluate scikit-learn huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
    

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
import random
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report

# Model required by assignment
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:

# Specific HF commit where parquet files exist
from datasets import load_dataset
COMMIT_ID = "6a6693e606d879b01e4cd97bd06181e6a932db08"

base_path = (
    f"hf://datasets/cardiffnlp/tweet_sentiment_multilingual@{COMMIT_ID}/english"
)

data_files = {
    "train": f"{base_path}/tweet_sentiment_multilingual-train.parquet",
    "validation": f"{base_path}/tweet_sentiment_multilingual-validation.parquet",
    "test": f"{base_path}/tweet_sentiment_multilingual-test.parquet",
}

dataset = load_dataset("parquet", data_files=data_files)

train_ds = dataset["train"]
test_ds = dataset["test"]
val_ds = dataset["validation"]

len(train_ds), len(test_ds), len(val_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)eet_sentiment_multilingual-train.parquet:   0%|          | 0.00/155k [00:00<?, ?B/s]

(…)entiment_multilingual-validation.parquet:   0%|          | 0.00/29.1k [00:00<?, ?B/s]

(…)weet_sentiment_multilingual-test.parquet:   0%|          | 0.00/64.8k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

(1839, 870, 324)

load model + tokenizer

In [ ]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token   # REQUIRED for LLaMA

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype="auto"
)

label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

def interpret_sentiment(raw_output):
    text = raw_output.lower().strip()
    words = text.replace(":", " ").replace("\n", " ").split()
    for w in words:
        if w in ["positive", "negative", "neutral"]:
            return w
    if text in ["positive", "negative", "neutral"]:
        return text
    return "neutral"

def evaluate_predictions(predict_fn, data_split):
    split = dataset[data_split]
    gold_labels = []
    pred_labels = []
    texts = []

    for idx, row in enumerate(split):
        if (idx + 1) % 60 == 0:
            print(f"Running inference on {idx+1}/{len(split)} items...")

        text = row["text"]
        gold_labels.append(row["label"])
        texts.append(text)

        pred = predict_fn(text)
        pred_labels.append(label2id[pred])

    print(classification_report(
        gold_labels,
        pred_labels,
        target_names=["negative", "neutral", "positive"]
    ))

    return texts, gold_labels, pred_labels


In [ ]:
prompt = "Classify the sentiment: I love this!"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Classify the sentiment: I love this! The design is so creative and unique, it's amazing how it turned out. The colors are vibrant


experiemnt 1 zero shot

In [ ]:
def predict_zero_shot(text):
    """
    Zero-shot sentiment prediction using a plain instruction prompt.
    The model receives only the task description + the tweet.
    """
    prompt = (
        "You are a sentiment classifier. "
        "Label the sentiment of the following text as positive, negative, or neutral.\n\n"
        f"Text: {text}\n"
        "Sentiment:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=4,
        do_sample=False,  # deterministic zero-shot
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    label = interpret_sentiment(decoded)
    return label

In [ ]:
print("=== Zero-shot Evaluation ===")
zs_texts, zs_true, zs_pred = evaluate_predictions(predict_zero_shot, data_split="test")
print("=== Zero-shot Evaluation Complete ===")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== Zero-shot Evaluation ===


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 60/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Running inference on 780/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 840/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

              precision    recall  f1-score   support

    negative       0.72      0.56      0.63       290
     neutral       0.47      0.70      0.56       290
    positive       0.77      0.57      0.65       290

    accuracy                           0.61       870
   macro avg       0.65      0.61      0.61       870
weighted avg       0.65      0.61      0.61       870

=== Zero-shot Evaluation Complete ===


Experiment 2. Few-shot in-context. learning

 We must get:

k=1: 1 positive + 1 neutral + 1 negative

k=3: 3 examples per class → 9 total examples

In [ ]:
import random

# Utility to pull k examples for a given label
def sample_examples(dataset_split, label, k):
    rows = [ex for ex in dataset_split if ex["label"] == label]
    return random.sample(rows, k)

# k = 1 exemplars
one_shot_examples = {
    "negative": sample_examples(train_ds, 0, 1),
    "neutral":  sample_examples(train_ds, 1, 1),
    "positive": sample_examples(train_ds, 2, 1),
}

# k = 3 exemplars
three_shot_examples = {
    "negative": sample_examples(train_ds, 0, 3),
    "neutral":  sample_examples(train_ds, 1, 3),
    "positive": sample_examples(train_ds, 2, 3),
}


def build_fewshot_prompt(text, examples_dict):
    """
    Construct a structured few-shot prompt containing examples
    followed by the query text.
    This version has improved formatting and stricter instructions.
    """
    sections = [
        "You are a sentiment classifier.",
        "Use the examples below to infer the correct label.",
        "Your output must be EXACTLY one word:",
        "positive, negative, or neutral.",
        ""
    ]

    # Add examples with cleaner formatting + separators
    for label, items in examples_dict.items():
        for ex in items:
            ex_text = ex["text"].strip()
            ex_label = ["negative", "neutral", "positive"][ex["label"]]

            sections.append("### Example")
            sections.append(f"Text: {ex_text}")
            sections.append(f"Label: {ex_label}")
            sections.append("")  # blank line for spacing

    # Add the query text
    sections.append("### Now classify the following text.")
    sections.append("Your answer MUST be exactly one word:")
    sections.append("positive, negative, or neutral.\n")
    sections.append(f"Text: {text.strip()}")
    sections.append("Label:")

    return "\n".join(sections)


In [ ]:
def predict_fewshot_k1(text):
    prompt = build_fewshot_prompt(text, one_shot_examples)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=3,
        do_sample=False,
        temperature=0.0
    )

    # Extract ONLY generated tokens (not the whole prompt)
    generated = output[0][inputs["input_ids"].shape[1]:]
    decoded = tokenizer.decode(generated, skip_special_tokens=True)

    return interpret_sentiment(decoded)


def predict_fewshot_k3(text):
    prompt = build_fewshot_prompt(text, three_shot_examples)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_new_tokens=3,
        do_sample=False,
        temperature=0.0
    )

    generated = output[0][inputs["input_ids"].shape[1]:]
    decoded = tokenizer.decode(generated, skip_special_tokens=True)

    return interpret_sentiment(decoded)



In [ ]:
print("=== Few-shot k = 1 ===")
k1_texts, k1_true, k1_pred = evaluate_predictions(predict_fewshot_k1, "test")
print("=== Few-shot k = 1 done ===")



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== Few-shot k = 1 ===


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 60/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Running inference on 780/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 840/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

              precision    recall  f1-score   support

    negative       0.85      0.34      0.49       290
     neutral       0.48      0.62      0.54       290
    positive       0.56      0.73      0.64       290

    accuracy                           0.57       870
   macro avg       0.63      0.57      0.56       870
weighted avg       0.63      0.57      0.56       870

=== Few-shot k = 1 done ===


In [ ]:
print("=== Few-shot k = 3 ===")
k3_texts, k3_true, k3_pred = evaluate_predictions(predict_fewshot_k3, "test")
print("=== Few-shot k = 3 done ===")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== Few-shot k = 3 ===


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 60/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Running inference on 780/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Running inference on 840/870 items...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

              precision    recall  f1-score   support

    negative       0.85      0.20      0.32       290
     neutral       0.54      0.13      0.21       290
    positive       0.39      0.97      0.55       290

    accuracy                           0.43       870
   macro avg       0.59      0.43      0.36       870
weighted avg       0.59      0.43      0.36       870

=== Few-shot k = 3 done ===


EXPERIMENT 3 — FINE-TUNING WITH LoRA

Prepare the dataset for training

Load the base model with PEFT/LoRA

Train for 2–3 epochs

Save the adapter

Load the adapter and evaluate on the test set

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

# Ensure pad token exists (REQUIRED FOR TRAINING)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

MAX_SEQ_LEN = 256

id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {v: k for k, v in id2label.items()}

def make_chat_style_example(example):
    gold_label = id2label[example["label"]]

    conversation = [
        {
            "role": "system",
            "content": (
                "You are a classifier for tweet sentiment. "
                "For each tweet you must answer with exactly one word: "
                "positive, neutral, or negative."
            ),
        },
        {
            "role": "user",
            "content": f"Tweet: {example['text']}\nSentiment:",
        },
        {
            "role": "assistant",
            "content": gold_label,
        },
    ]

    chat_text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=False,
    )

    tokenized = tokenizer(
        chat_text,
        max_length=MAX_SEQ_LEN,
        truncation=True,
    )

    return tokenized

# Tokenize dataset
tokenized_train = dataset["train"].map(
    make_chat_style_example,
    remove_columns=dataset["train"].column_names,
)

tokenized_val = dataset["validation"].map(
    make_chat_style_example,
    remove_columns=dataset["validation"].column_names,
)

# Shuffle for training stability
tokenized_train = tokenized_train.shuffle(seed=SEED)
tokenized_val = tokenized_val.shuffle(seed=SEED)

tokenized_train[0]


Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

{'input_ids': [128000,
  128000,
  128006,
  9125,
  128007,
  271,
  38766,
  1303,
  33025,
  2696,
  25,
  6790,
  220,
  2366,
  18,
  198,
  15724,
  2696,
  25,
  220,
  845,
  4723,
  220,
  2366,
  20,
  271,
  2675,
  527,
  264,
  34465,
  369,
  12072,
  27065,
  13,
  1789,
  1855,
  12072,
  499,
  2011,
  4320,
  449,
  7041,
  832,
  3492,
  25,
  6928,
  11,
  21277,
  11,
  477,
  8389,
  13,
  128009,
  128006,
  882,
  128007,
  271,
  49462,
  25,
  57189,
  6170,
  389,
  3293,
  71389,
  315,
  9349,
  25,
  5751,
  586,
  690,
  320,
  66007,
  819,
  8,
  1063,
  1274,
  369,
  433,
  389,
  320,
  68,
  2635,
  1938,
  8,
  4723,
  13,
  220,
  16,
  720,
  32458,
  3904,
  25,
  128009,
  128006,
  78191,
  128007,
  271,
  43324,
  128009],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [ ]:

# LLaMA models often don’t define a pad token.
# We reuse the EOS token so batching works properly.
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    print("Tokenizer had no PAD token → using EOS as PAD.")

# Some models require explicit config updates
if hasattr(model, "config"):
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"Padding token confirmed → '{tokenizer.pad_token}' (ID={tokenizer.pad_token_id})")

Padding token confirmed → '<|eot_id|>' (ID=128009)


In [ ]:
import os

# === Disable external logging tools ===
os.environ["WANDB_DISABLED"] = "true"

from transformers import (
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
from peft import PeftModel, get_peft_model, LoraConfig

# 1) Language-modeling collator
lm_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# 2) LoRA configuration
lora_settings = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

# 3) Avoid stacking LoRA layers repeatedly
if isinstance(model, PeftModel):
    base_llm = model.base_model
else:
    base_llm = model

model_with_lora = get_peft_model(base_llm, lora_settings)
model_with_lora.print_trainable_parameters()

# 4) TrainingArguments
train_cfg = TrainingArguments(
    output_dir="./sentiment_lora_llama32",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=False,
    report_to="none",
    fp16=torch.cuda.is_available(),  # good for T4
    save_total_limit=1,
)


# 5) Trainer object
trainer = Trainer(
    model=model_with_lora,
    args=train_cfg,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=lm_collator,
)

print("Trainer prepared.")

The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750
Trainer prepared.


In [ ]:
print("Starting LoRA fine-tuning...")

try:
    output = trainer.train()
    print("Training completed successfully!")
except RuntimeError as err:
    print("\nRuntimeError detected during training:\n", err)
    print("\nIf this mentions CUDA or device-side assert, tell me and I’ll fix your exact issue.")

Starting LoRA fine-tuning...


Step,Training Loss
50,1.949200
100,1.501000
150,1.463700
200,1.464100
250,1.447200
300,1.406200
350,1.387800
400,1.401000
450,1.403300
500,1.340700


Training completed successfully!


In [ ]:
print("Evaluating on validation set...")
val_metrics = trainer.evaluate(eval_dataset=tokenized_val)
print(val_metrics)


Evaluating on validation set...


{'eval_loss': 1.4196717739105225, 'eval_runtime': 11.0642, 'eval_samples_per_second': 29.284, 'eval_steps_per_second': 3.706, 'epoch': 3.0}


In [ ]:
# Use the LoRA-augmented model produced by training
lora_model = trainer.model
lora_model.eval()

def classify_sentiment_lora(tweet: str) -> str:
    """
    Produce a sentiment label ('positive', 'neutral', 'negative')
    using the LoRA-fine-tuned LLaMA model.
    """

    # Reformulated system instruction
    system_instruction = (
        "Determine the sentiment expressed in a tweet. "
        "Only reply using one word: positive, neutral, or negative."
    )

    # Conversation layout
    dialogue = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": f"The tweet is:\n{tweet}\n\nSentiment?"},
    ]

    # Build prompt
    chat_prompt = tokenizer.apply_chat_template(
        dialogue,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize prompt
    encoded_input = tokenizer(
        chat_prompt,
        return_tensors="pt"
    ).to(lora_model.device)

    # Generate continuation only
    with torch.no_grad():
        output = lora_model.generate(
            **encoded_input,
            max_new_tokens=5,
            temperature=0.0,
            do_sample=False
        )

    # Extract ONLY generated tokens
    start = encoded_input["input_ids"].shape[-1]
    continuation_ids = output[0][start:]
    continuation_text = tokenizer.decode(
        continuation_ids,
        skip_special_tokens=True
    ).strip()

    # Normalize to {positive, neutral, negative}
    return interpret_sentiment(continuation_text)



In [ ]:
# -----------------------------------------
# Label parser (improved)
# -----------------------------------------
def parse_label_from_output(text):
    """
    Convert the raw generated text into one of the three valid labels.
    Handles capitalization, punctuation, and extra words.
    """
    cleaned = text.strip().lower()

    # Direct keyword detection
    if "positive" in cleaned:
        return "positive"
    if "negative" in cleaned:
        return "negative"
    if "neutral" in cleaned:
        return "neutral"

    # Look at first word if model adds chatter
    first = cleaned.split()[0] if cleaned else ""

    if first.startswith("pos"):
        return "positive"
    if first.startswith("neg"):
        return "negative"
    if first.startswith("neu"):
        return "neutral"

    # Worst-case fallback → neutral (safest default)
    return "neutral"


# -----------------------------------------
# Evaluate the fine-tuned LoRA model on the test split
# -----------------------------------------
def evaluate_lora_model():
    """
    Runs the fine-tuned (LoRA) model across the entire test set,
    collects predictions, and prints out a standard classification report.
    """

    test_split = dataset["test"]
    gold_labels = []
    pred_labels = []
    raw_texts = []

    for idx, example in enumerate(test_split):

        text = example["text"]
        raw_texts.append(text)

        # Convert numeric label → string label
        true_label = id2label[example["label"]]
        gold_labels.append(true_label)

        # Model prediction
        model_output = classify_sentiment_lora(text)
        pred_labels.append(model_output)

        # progress display every 50 examples
        if (idx + 1) % 50 == 0:
            print(f"Processed {idx+1}/{len(test_split)} examples...")

    print("\n=== LoRA Fine-Tuned Model Results ===")
    print(classification_report(
        gold_labels,
        pred_labels,
        labels=["negative", "neutral", "positive"],
        digits=3
    ))

    return raw_texts, gold_labels, pred_labels


# Run LoRA evaluation
ft_texts, ft_gold, ft_pred = evaluate_lora_model()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 50/870 examples...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 100/870 examples...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Processed 800/870 examples...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 850/870 examples...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for


=== LoRA Fine-Tuned Model Results ===
              precision    recall  f1-score   support

    negative      0.759     0.772     0.766       290
     neutral      0.589     0.648     0.617       290
    positive      0.805     0.710     0.755       290

    accuracy                          0.710       870
   macro avg      0.718     0.710     0.713       870
weighted avg      0.718     0.710     0.713       870



In [ ]:
import pandas as pd

def build_error_dataframe(texts, gold, pred):
    """
    Create a pandas DataFrame containing:
        - tweet text
        - true label
        - predicted label
        - correctness flag
    """
    df = pd.DataFrame({
        "text": texts,
        "gold": gold,
        "pred": pred,
    })
    df["correct"] = df["gold"] == df["pred"]
    return df


# Build error analysis dataframe for LoRA model
error_df = build_error_dataframe(ft_texts, ft_gold, ft_pred)

# Show summary
print("Total examples:", len(error_df))
print("Correct predictions:", error_df["correct"].sum())
print("Incorrect predictions:", (~error_df["correct"]).sum())


# ---------------------------------------
# Misclassified examples grouped by type
# ---------------------------------------
misclassified = error_df[~error_df["correct"]]

print("\n=== Misclassified Examples by True Label ===")
for label in ["negative", "neutral", "positive"]:
    subset = misclassified[misclassified["gold"] == label]
    print(f"\nTrue label = {label}  |  Count = {len(subset)}")
    if len(subset) > 0:
        display(subset.head(5))   # Show first 5 errors for that class


Total examples: 870
Correct predictions: 618
Incorrect predictions: 252

=== Misclassified Examples by True Label ===

True label = negative  |  Count = 66


,text,gold,pred,correct
12,"Iraqi Forces set to storm 3 areas of #Mosul, #...",negative,neutral,False
15,Electoral College must reject Trump #rejecttru...,negative,neutral,False
27,Ben Carson for Housing & Urban Development?? 😐...,negative,neutral,False
63,Donald Trump does not have a clue about global...,negative,neutral,False
72,"@user By this logic, we shouldn't have lifted ...",negative,neutral,False



True label = neutral  |  Count = 102


,text,gold,pred,correct
7,Sentinel Editorial: FBI’s Comey ‘had no one of...,neutral,negative,False
10,Presenting the Red Line symptoms of Aconite in...,neutral,positive,False
16,Git 'em girls #BarackObama #blm #blacklivesmat...,neutral,positive,False
46,I'm crying over Richard and Leonard Cohen 😭😭😭 ...,neutral,negative,False
58,First Look at Neighbors 2 with Zac Efron Shirt...,neutral,positive,False



True label = positive  |  Count = 84


,text,gold,pred,correct
8,Samsung to Bring Android 7.0 Nougat to Galaxy ...,positive,neutral,False
17,The Reputation Doctor weighs in on Tony Romo #...,positive,neutral,False
32,I've been listening to Leonard Cohen nonstop f...,positive,negative,False
35,Listen to #NBAwards Winner @user interview on ...,positive,neutral,False
38,Kelly Anne Conway is here reporting the wall i...,positive,neutral,False
